# TMNetV2

In [1]:
import os
import pathlib
import tensorflow as tf
import pandas as pd
import wandb

from bcd.model.network.base import NetworkConfig
from bcd.model.network.tmnetv2 import TMNetV2Config, TMNetV2Factory
from bcd.model.repo import ExperimentRepo
from bcd.model.base import *
from bcd.model.experiment import FeatureExtractionExperiment
from bcd.model.config import *

pd.set_option('display.max_rows',999)

## Configuration

In [2]:
# Experiment Parameters
%env "WANDB_NOTEBOOK_NAME" "tmnetv2.ipynb"
mode = "Stage"
force = False  # Whether to retrain if the model and config already exists.
base_models = [DenseNet(), EfficientNet(), Inception(), InceptionResNet(), MobileNet(), ResNet(), Xception()]

env: "WANDB_NOTEBOOK_NAME"="tmnetv2.ipynb"


In [3]:
def create_config(network_config: NetworkConfig):
    project_config = ProjectConfig(mode=mode)

    train_config = TrainConfig(epochs=50, learning_rate=1e-4)    

    dataset_config = DatasetConfig(mode=mode)

    checkpoint_config = CheckPointConfig(monitor="val_accuracy", verbose=1, save_best_only=True, save_weights_only=False, mode="auto")

    early_stop_config = EarlyStopConfig(min_delta=1e-4, monitor="val_loss", patience=10, restore_best_weights=True, verbose=1)

    learning_rate_schedule_config = LearningRateScheduleConfig(min_delta=1e-4, monitor="val_loss", factor=0.5, patience=3, restore_best_weights=True, verbose=1, mode="auto")

    config = Config(project=project_config, 
                    dataset=dataset_config, 
                    train=train_config, 
                    network=network_config, 
                    checkpoint=checkpoint_config, 
                    early_stop=early_stop_config, 
                    learning_rate_schedule=learning_rate_schedule_config)
    return config

network_config = TMNetV2Config(activation="sigmoid", input_shape=(224,224,3), output_shape=1, dense1=1024, dropout1=0.5, dense2=1024, dropout2=0.5)
config = create_config(network_config=network_config)

## Load Data

In [4]:
train_dir = pathlib.Path(config.dataset.train_dir).with_suffix('') 
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

# Validation DataSet (10%)
val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels="inferred",
    color_mode="rgb",
    image_size=(224,224),
    shuffle=True,
    validation_split=0.2,
    subset='validation',
    interpolation="bilinear",
    seed=123,
    batch_size=config.dataset.batch_size)

Found 816 files belonging to 2 classes.
Using 653 files for training.
Found 816 files belonging to 2 classes.
Using 163 files for validation.


## Callbacks

In [5]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=config.early_stop.monitor, 
                                                       min_delta=config.early_stop.min_delta,
                                                       patience=config.early_stop.patience, 
                                                       restore_best_weights=config.early_stop.restore_best_weights,
                                                       verbose=config.early_stop.verbose)

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=config.learning_rate_schedule.monitor,
                                                          factor=config.learning_rate_schedule.factor,
                                                          patience=config.learning_rate_schedule.patience,
                                                          verbose=config.learning_rate_schedule.verbose,
                                                          mode=config.learning_rate_schedule.mode,
                                                          min_delta=config.learning_rate_schedule.min_delta,
                                                          min_lr=config.learning_rate_schedule.min_lr)
callbacks = [early_stop_callback, reduce_lr_callback]

## Dependencies

In [6]:
repo = ExperimentRepo(mode = mode, project=config.project.name)
optimizer=tf.keras.optimizers.Adam
metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]

## Build Factory

In [7]:
factory = TMNetV2Factory(config=config.network)

## Run Experiments

In [8]:
for base_model in base_models:
    network = factory.create(base_model=base_model)
    experiment = FeatureExtractionExperiment(network=network, config=config, optimizer=optimizer, repo=repo, callbacks=callbacks, metrics=metrics, tags=[network.architecture, base_model.name], force=False)
    experiment.run(train_ds=train_ds, val_ds=val_ds)


# ================================================================================================ #
                                          TMNetV2_DenseNet                                          
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 TMNetV2_DenseNet_input_laye  [(None, 224, 224, 3)]    0         
 r (InputLayer)                                                  
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 tf.math.truediv_1 (TF

wandb: Currently logged in as: aistudio. Use `wandb login --relogin` to force relogin


Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 0.7541 - accuracy: 0.5391 - auc: 0.5466 - precision: 0.4684 - recall: 0.3881
Epoch 1: val_accuracy improved from -inf to 0.63804, saving model to models/stage/TMNetV2_DenseNet-knjvzvub.keras
21/21 [==============================] - 131s 5s/step - loss: 0.7541 - accuracy: 0.5391 - auc: 0.5466 - precision: 0.4684 - recall: 0.3881 - val_loss: 0.6681 - val_accuracy: 0.6380 - val_auc: 0.5718 - val_precision: 0.5366 - val_recall: 0.3548 - lr: 1.0000e-04
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 0.6737 - accuracy: 0.5972 - auc: 0.6512 - precision: 0.5481 - recall: 0.4580
Epoch 2: val_accuracy did not improve from 0.63804
21/21 [==============================] - 108s 5s/step - loss: 0.6737 - accuracy: 0.5972 - auc: 0.6512 - precision: 0.5481 - recall: 0.4580 - val_loss: 0.6576 - val_accuracy: 0.6196 - val_auc: 0.6160 - val_precision: 0.5000 - val_recall: 0.3387 - lr: 1.0000e-04
Epoch 3/50
21/21 [=====

wandb: Waiting for artifact run-knjvzvub-TMNetV2_DenseNet-knjvzvub.keras to be committed...



wandb: Committed artifact aistudio/Breast-Cancer-Detection-Stage/run-knjvzvub-TMNetV2_DenseNet-knjvzvub.keras:v0
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

# ================================================================================================ #
                                          TMNetV2_Xception                                          
# ------------------------------------------------------------------------------------------------ #
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 TMNetV2_Xception_input_laye  [(None, 224, 224, 3)]    0         
 r (InputLayer)                                                  
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                      

Epoch 1/50
21/21 [==============================] - ETA: 0s - loss: 0.8583 - accuracy: 0.5253 - auc: 0.5448 - precision: 0.4714 - recall: 0.4023
Epoch 1: val_accuracy did not improve from 0.71779

Epoch 1: val_accuracy did not improve from 0.71779

Epoch 1: val_accuracy did not improve from 0.71779

Epoch 1: val_accuracy did not improve from 0.71779

Epoch 1: val_accuracy did not improve from 0.67485

Epoch 1: val_accuracy did not improve from 0.67485

Epoch 1: val_accuracy improved from -inf to 0.39877, saving model to models/stage/TMNetV2_Xception-n2ihi5t8.keras
21/21 [==============================] - 85s 4s/step - loss: 0.8583 - accuracy: 0.5253 - auc: 0.5448 - precision: 0.4714 - recall: 0.4023 - val_loss: 0.7162 - val_accuracy: 0.3988 - val_auc: 0.5473 - val_precision: 0.3784 - val_recall: 0.9032 - lr: 1.0000e-04
Epoch 2/50
21/21 [==============================] - ETA: 0s - loss: 0.7728 - accuracy: 0.5804 - auc: 0.5925 - precision: 0.5242 - recall: 0.4545
Epoch 2: val_accuracy di

wandb: Waiting for artifact run-n2ihi5t8-TMNetV2_Xception-n2ihi5t8.keras to be committed...



wandb: Committed artifact aistudio/Breast-Cancer-Detection-Stage/run-n2ihi5t8-TMNetV2_Xception-n2ihi5t8.keras:v0
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch/accuracy,▁▁▂▂▃▃▃▃▄▄▅▅▄▄▄▄▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██▇▇██▇▇
epoch/auc,▁▁▂▂▃▃▄▄▄▄▅▅▅▅▅▅▆▆▇▇▇▇▆▆▇▇██▇▇▇▇████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
epoch/learning_rate,████████████████████████▄▄▄▄▄▄▂▂▂▂▂▂▁▁▁▁
epoch/loss,██▇▇▆▆▆▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▁▁▂▂▂▂▁▁▁▁▁▁▂▂
epoch/lr,██████████████████████████▄▄▄▄▄▄▂▂▂▂▂▂▁▁
epoch/precision,▁▁▂▂▃▃▃▃▄▄▅▅▄▄▄▄▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██▇▇██▇▇
epoch/recall,▁▁▂▂▄▄▄▄▄▄▅▅▄▄▄▄▆▆▇▇▆▆▆▆▇▇▇▇▇▇▇▇████████
epoch/val_accuracy,▁▁▆▆▅▅▆▆▆▆▇▇▆▆▆▆▇▇██▇▇▇▇██████▆▆▇▇▇▇▇▇▇▇
epoch/val_auc,▃▃▂▂▁▁▃▃▃▃▃▃▄▄▃▃▆▆▆▆▆▆▅▅▅▅▆▆▇▇▆▆▇▇▇▇████
epoch/val_loss,██▃▃▄▄▃▃▃▃▃▃▃▃▃▃▂▂▁▁▄▄▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆
